In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

#Define the column names.
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm", "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                          skiprows=1,
                          names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                      skiprows=1,
                      names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                            skiprows=1,
                            names=COLUMNS)
#Defining FeatureColumns 
feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]

#Creating the Regressor
regressor = tf.estimator.DNNRegressor(feature_columns=feature_cols,
                                     hidden_units=[10,10],
                                     model_dir="/tmp/boston_model")
#Building the input_fn
def get_input_fn(data_set, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=pd.DataFrame({k: data_set[k].values for k in FEATURES}),
        y=pd.Series(data_set[LABEL].values),
        num_epochs=num_epochs,
        shuffle=shuffle)

#Training the Regressor
regressor.train(input_fn=get_input_fn(training_set), steps=5000)

#Evaluating the Model
ev = regressor.evaluate(input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))

#Retrieve the loss fom the ev results and print it to output.
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

#Making Predictions.
y = regressor.predict(input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False))
# . predict() returns an iterator of dicts.
#convert to a list and print predictions.
predictions = list(p["predictions"] for p in itertools.islice(y, 6))
print("Predictions: {}".format(str(predictions)))




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/boston_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/boston_model/model.ckpt-5000
INFO:tensorflow:Saving checkpoints for 5001 into /tmp/boston_model/model.ckpt.
INFO:tensorflow:loss = 4358.36, step = 5001
INFO:tensorflow:global_step/sec: 320.457
INFO:tensorflow:loss = 4582.15, step = 5101 (0.315 sec)
INFO:tensorflow:global_step/sec: 328.007
INFO:tensorflow:loss = 4395.4, step = 5201 (0.302 sec)
INFO:tensorflow:global_step/sec: 314.698
INFO:tensorflow:loss = 3665.17, step = 5301 (0.319 sec)
INFO:tensorflow:global_step/sec: 351.049
INFO:tensorflow:loss = 4556.94, step = 5401 (0.285 sec)
INFO:tensorflow:global_step/s